<a href="https://colab.research.google.com/github/YoungsikMoon/FORS/blob/main/%EA%B8%B0%EC%9D%80%EC%84%9C/FORS_%EA%B8%B0%EC%9D%80%EC%84%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터프레임 엑셀파일로 전환 예제 코드


예시 데이터 폼

In [ ]:
date = { name : [name.wav, 'eunseo'], old : [old.wav, 'age'] , location : [location.wav, 'seoul']}

데이터를 엑셀로 전환

In [ ]:
import pandas as pd

# 제공된 데이터
data = {
    "name": ["name.wav", "eunseo"],
    "old": ["old.wav", "age"],
    "location": ["location.wav", "seoul"]
}

# 데이터 프레임을 생성하기 위한 빈 딕셔너리
colum = {
    "Attribute": [],
    "File": [],
    "Value": []
}

# 딕셔너리를 순회하면서 데이터 프레임용 데이터 추출
for key, value in data.items():
    colum["Attribute"].append(key)
    colum["File"].append(value[0])
    colum["Value"].append(value[1])

# 판다스 데이터 프레임 생성
df = pd.DataFrame(colum)
df

# 주석 처리된 함수 호출 (실행 방지)
# print(df)

In [ ]:
# 엑셀 파일로 저장하기 위한 코드
# 라이브러리 로드
from openpyxl import Workbook

# 데이터 프레임을 엑셀 파일로 저장 (예: output.xlsx)
excel_filepath = '/content/Attribute_Data.xlsx'
df.to_excel(excel_filepath, index=False)

# 주석 처리된 함수 호출 (실행 방지)
# print("Excel file has been created at:", excel_filepath)


# Meta - MMS(Massively Multilingual Speech) 연구


다운로드 과정이 오래걸리고, 코드가 복잡하다

참고자료 : [원본파일](https://github.com/facebookresearch/fairseq/tree/main/examples/mms?utm_source=pytorchkr)

Colab에서 MMS-ASR 추론 실행하기

이 노트북에서는 MMS ASR 모델을 사용하여 간단한 ASR 추론을 실행하는 방법에 대한 예제를 제공합니다.

epk2112[깃허브](https://github.com/epk2112/fairseq_meta_mms_Google_Colab_implementation) 제공

## 1단계: fairseq-py를 복제하고 최신 버전을 설치합니다.

In [1]:
!mkdir "temp_dir" #임시데이터 중간결과 저장하기 위한 디렉토리 생성
!git clone https://github.com/pytorch/fairseq #저장소 복제

# Change current working directory
!pwd #현재 작업중인 디렉토리 경로 출력
%cd "/content/fairseq" #지속적으로 디렉토리 현재 위치 변경 유지
!pip install --editable ./ #개발모드에서 수정되면 패키지에도 반영
!pip install tensorboardX #모델의 학습 과정 시각화 사용 라이브러리

Cloning into 'fairseq'...
remote: Enumerating objects: 35184, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 35184 (delta 61), reused 72 (delta 47), pack-reused 35079
Receiving objects: 100% (35184/35184), 25.22 MiB | 14.37 MiB/s, done.
Resolving deltas: 100% (25548/25548), done.
/content
[Errno 2] No such file or directory: '/content/fairseq #지속적으로 디렉토리 현재 위치 변경 유지'
/content
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.4 MB/s eta 0:00:00


## 2. MMS 모델 다운로드


다운로드할 선호 모델을 주석 해제하세요. 이 예제에서는 시연 목적으로 MMS-FL102를 사용합니다. 더 나은 모델 품질과 언어 커버리지를 위해 MMS-1B-ALL 모델을 사용할 수 있지만, 더 많은 RAM이 필요하므로 Colab-Free 대신 Colab-Pro를 사용하세요.

In [2]:
# MMS-1B:FL102 model - 102 Languages - FLEURS Dataset
!wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_fl102.pt'
#wget은 웹에서 파일을 다운로드하기 위해 사용되는 프로그램

# # MMS-1B:L1107 - 1107 Languages - MMS-lab Dataset
# !wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_l1107.pt'

# # MMS-1B-all - 1162 Languages - MMS-lab + FLEURS + CV + VP + MLS
# !wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_all.pt'

--2024-04-25 10:00:39--  https://dl.fbaipublicfiles.com/mms/asr/mms1b_fl102.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.25, 13.226.210.15, 13.226.210.78, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4851043301 (4.5G) [binary/octet-stream]
Saving to: ‘./models_new/mms1b_fl102.pt’

mms1b_fl102.pt      100%[===================>]   4.52G   196MB/s    in 33s     

2024-04-25 10:01:12 (140 MB/s) - ‘./models_new/mms1b_fl102.pt’ saved [4851043301/4851043301]



## 3. 오디오 파일 준비

'/content/audio_samples/' 경로에 폴더를 만들고 트랜스 크립 션해야 하는 .wav 오디오 파일(예: '/content/audio_samples/audio.wav')을 업로드합니다.

참고: 사용 중인 오디오 데이터의 샘플 속도가 16kHz인지 확인해야 합니다. .mp3 파일을 .wav로 변환하고 오디오 샘플 속도를 고정하는 아래 예제와 같이 FFMPEG를 사용하면 쉽게 이 작업을 수행할 수 있습니다.

여기서는 예제로 FLEURS 영어 MP3 오디오를 사용합니다.

In [3]:
!wget -P ./audio_samples/ 'https://datasets-server.huggingface.co/assets/google/fleurs/--/en_us/train/0/audio/audio.mp3'
#wget 인터넷에서 파일 다운로드 하는데 사용
#-P 옵션은 다운로드한 파일을 저장할 디렉터리 지정

!ffmpeg -y -i ./audio_samples/audio.mp3 -ar 16000 ./audio_samples/audio.wav
#ffmpeg은 파일 형식을 변환하거나, 비디오를 편집하거나, 스트림을 다루는 등 다양한 미디어 작업에 사용되는 명령어
#-y 옵션은 출력 파일이 이미 존재할 경우 덮어쓰기 강제
#-i 입력 파일 지정
#ar 16000 -> 옵션은 오디오의 샘플 레이트를 16000 Hz로 설정합니다. 샘플 레이트는 초당 오디오 샘플 수를 의미하며, 이는 오디오의 품질과 파일 크기에 영향을 미칩니다.

#즉 위 명령어는 온라인에서 audio.mp3를 ./audio_samples/ 디렉토리에 다운 받았고,  16000hz로 설정해서 ./audio_samples/audio.wav로 전환했다.

--2024-04-25 10:01:13--  https://datasets-server.huggingface.co/assets/google/fleurs/--/en_us/train/0/audio/audio.mp3
Resolving datasets-server.huggingface.co (datasets-server.huggingface.co)... 13.226.228.82, 13.226.228.19, 13.226.228.70, ...
Connecting to datasets-server.huggingface.co (datasets-server.huggingface.co)|13.226.228.82|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-04-25 10:01:13 ERROR 403: Forbidden.

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig -

## 4: 추론을 실행하고 오디오를 트랜스크립션합니다.

아래 예에서는 영어로 된 문장을 텍스트로 변환합니다.

다른 언어를 전사하려면

MMS README ASR 섹션으로 이동합니다.
지원 언어 링크를 엽니다.
언어 이름 열을 기준으로 대상 언어를 찾습니다.
해당 Iso 코드를 복사합니다.
--lang "eng"를 새 Iso 코드로 바꾸기
전사 품질을 향상시키기 위해 사용자는 다음 지침에 따라 언어 모델 (LM) 디코딩을 사용할 수 있습니다. ASR LM 디코딩

In [4]:
import os #파이썬 모듈 불러옴

os.environ["TMPDIR"] = '/content/temp_dir'
#임시파일을 저장하기 위한 디렉토리 설정, 데이터를 저장할 위치 지정
os.environ["PYTHONPATH"] = "."
#검색할 경로를 현재 디렉토리 설정, 디렉토리에 있는 모듈 쉽게 임포트할 수 있게 해줌
os.environ["PREFIX"] = "INFER"
#로깅이나 다른 구성 설정에서 사용될 수 있음
os.environ["HYDRA_FULL_ERROR"] = "1"
#Hydra 설정 관리 라이브러리에서 전체 오류 메시지를 활성화하는데 사용됨
os.environ["USER"] = "micro"
#사용자 이름을 'micro'를 설정

!python examples/mms/asr/infer/mms_infer.py --model "/content/fairseq/models_new/mms1b_fl102.pt" --lang "eng" --audio "/content/fairseq/audio_samples/audio.wav"
#지정된 경로에 있는 mms_infer.py라는 파이썬 스크립트 실행
#--model 모델 파일의 경로를 인자로 전달, 인식을 수행할 때 사용한 신경망 모델
#--lang "eng" 스크립트 언어 옵션 전달, 인식 과정에서 사용할 영어로 지정
#--audio 처리할 오디오 파일의 경로를 인자로 제공

python3: can't open file '/content/examples/mms/asr/infer/mms_infer.py': [Errno 2] No such file or directory


## 5: 언어 모델을 사용한 빔 검색 디코딩 및 오디오 파일 트랜스크립션

MMS는 CTC 모델이므로, 언어 모델을 사용하여 빔 서치 디코딩을 실행함으로써 정확도를 더욱 향상시킬 수 있습니다.

아직 MMS에서 사용된 언어 모델을 오픈 소스화하지 않았지만, 우리는 논문의 부록 섹션에서 언어 모델을 훈련시키기 위해 사용된 데이터와 명령어의 세부사항을 제공했습니다.

이 튜토리얼에서는, Common Crawl 데이터를 기반으로 하고 Likhomanenko, Tatiana 등이 "ASR 평가를 다시 생각하다: 우리의 모델은 충분히 강력한가?"라는 논문을 통해 공개적으로 제공된 대체 영어 언어 모델을 사용할 것입니다. 언어 모델은 여기 GitHub 저장소에서 접근할 수 있습니다.

In [5]:
!mkdir -p /content/lmdecode

!wget -P /content/lmdecode  https://dl.fbaipublicfiles.com/wav2letter/rasr/tutorial/lm_common_crawl_small_4gram_prun0-6-15_200kvocab.bin # smaller LM
!wget -P /content/lmdecode  https://dl.fbaipublicfiles.com/wav2letter/rasr/tutorial/lexicon.txt

--2024-04-25 10:01:14--  https://dl.fbaipublicfiles.com/wav2letter/rasr/tutorial/lm_common_crawl_small_4gram_prun0-6-15_200kvocab.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.25, 13.226.210.111, 13.226.210.78, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2627163608 (2.4G) [application/octet-stream]
Saving to: ‘/content/lmdecode/lm_common_crawl_small_4gram_prun0-6-15_200kvocab.bin’

lm_common_crawl_sma 100%[===================>]   2.45G  56.0MB/s    in 47s     

2024-04-25 10:02:02 (53.1 MB/s) - ‘/content/lmdecode/lm_common_crawl_small_4gram_prun0-6-15_200kvocab.bin’ saved [2627163608/2627163608]

--2024-04-25 10:02:02--  https://dl.fbaipublicfiles.com/wav2letter/rasr/tutorial/lexicon.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.111, 13.226.210.15, 13.226.210.78, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicf

In [6]:
# 필수 개발 라이브러리 설치
! sudo apt-get install -y libfftw3-dev libsndfile1-dev libgoogle-glog-dev libopenmpi-dev libboost-all-dev
# 설명:
# - libfftw3-dev: FFTW 라이브러리 (고속 푸리에 변환 알고리즘 지원)의 개발 파일 설치.
# - libsndfile1-dev: 오디오 파일을 다루는 라이브러리의 개발 파일 설치.
# - libgoogle-glog-dev: 구글의 로깅 라이브러리 개발 파일 설치.
# - libopenmpi-dev: 고성능 병렬 컴퓨팅을 위한 MPI 구현 라이브러리의 개발 파일 설치.
# - libboost-all-dev: Boost 라이브러리 (C++ 프로그래밍 향상을 위한 라이브러리 집합) 설치.

# KenLM 언어 모델 도구 설치
! cd /tmp && git clone https://github.com/kpu/kenlm && cd kenlm && mkdir build && cd build && cmake .. -DCMAKE_BUILD_TYPE=Release && make install -j$(nproc)
# 설명:
# - /tmp 디렉터리로 이동
# - KenLM 저장소를 클론
# - 빌드 디렉터리 생성 후 해당 디렉터리로 이동
# - CMake를 사용하여 릴리즈 모드로 프로젝트 구성
# - 병렬로 빌드 및 설치 실행 (사용 가능한 모든 CPU 코어 사용)

# Intel MKL 설치를 위한 GPG 키 다운로드 및 추가
! cd /tmp && wget https://apt.repos.intel.com/intel-gpg-keys/GPG-PUB-KEY-INTEL-SW-PRODUCTS-2019.PUB && \
    apt-key add GPG-PUB-KEY-INTEL-SW-PRODUCTS-2019.PUB
# 설명:
# - /tmp 디렉터리로 이동
# - Intel의 공개 GPG 키 다운로드
# - 다운로드한 키를 apt 키 리스트에 추가하여 패키지의 유효성 검증

# Intel MKL 저장소 추가 및 패키지 설치
! sh -c 'echo deb https://apt.repos.intel.com/mkl all main > /etc/apt/sources.list.d/intel-mkl.list' && \
    apt-get update && DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends intel-mkl-64bit-2020.0-088
# 설명:
# - Intel MKL의 APT 저장소를 시스템의 패키지 소스 리스트에 추가
# - 패키지 리스트 업데이트
# - 사용자 입력 없이 Intel MKL 패키지 설치 (비대화형 설치 옵션 사용)

# 기존에 설치된 MKL 라이브러리 제거
! rm -rf /usr/local/lib/libmkl*
# 설명:
# - /usr/local/lib 디렉터리 내의 모든 MKL 관련 라이브러리 파일 삭제
# - 이는 버전 충돌이나 중복 링크 문제를 방지하기 위함입니다.

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libboost-all-dev is already the newest version (1.74.0.3ubuntu7).
libopenmpi-dev is already the newest version (4.1.2-2ubuntu1).
libsndfile1-dev is already the newest version (1.0.31-2ubuntu0.1).
The following additional packages will be installed:
  libfftw3-bin libfftw3-double3 libfftw3-long3 libfftw3-quad3 libfftw3-single3
  libgflags-dev libgflags2.2 libgoogle-glog0v5 libunwind-dev
Suggested packages:
  libfftw3-doc
The following NEW packages will be installed:
  libfftw3-bin libfftw3-dev libfftw3-double3 libfftw3-long3 libfftw3-quad3
  libfftw3-single3 libgflags-dev libgflags2.2 libgoogle-glog-dev
  libgoogle-glog0v5 libunwind-dev
0 upgraded, 11 newly installed, 0 to remove and 45 not upgraded.
Need to get 6,861 kB of archives.
After this operation, 32.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-double3 amd64 3.3.8-2ubunt

In [7]:
# 'flashlight' 디렉터리가 존재하는 경우, 이를 삭제하여 깨끗한 설치 환경을 준비
! rm -rf flashlight
# 설명:
# - 현재 디렉터리에서 'flashlight' 디렉터리와 그 내부의 모든 파일과 하위 디렉터리를 강제로 삭제

# 'flashlight' 프로젝트의 Git 리포지터리 클론, 서브모듈 포함
! git clone --recursive https://github.com/flashlight/flashlight.git
# 설명:
# - Git을 사용해 'flashlight' 리포지터리를 재귀적으로 클론, 필요한 모든 서브모듈 포함

# 클론된 'flashlight' 디렉터리로 이동
%cd flashlight
# 설명:
# - 현재 작업 디렉터리를 'flashlight'로 변경

# 특정 커밋으로 체크아웃
! git checkout 035ead6efefb82b47c8c2e643603e87d38850076
# 설명:
# - 'flashlight' 프로젝트의 특정 커밋으로 체크아웃하여, 이 특정 버전의 코드로 작업

# Python 바인딩 디렉터리로 이동
%cd bindings/python
# 설명:
# - 'flashlight' 프로젝트 내의 Python 바인딩을 포함하는 디렉터리로 이동

# Python 바인딩 설치
! python3 setup.py install
# 설명:
# - 'setup.py' 스크립트를 실행하여 Python 환경에 'flashlight' 바인딩 설치

# '/content/fairseq' 디렉터리로 이동
%cd /content/fairseq
# 설명:
# - 작업 디렉터리를 '/content/fairseq'로 변경, 이 디렉터리는 다른 프로젝트나 라이브러리 작업에 사용될 수 있음


Cloning into 'flashlight'...
remote: Enumerating objects: 25902, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 25902 (delta 32), reused 71 (delta 19), pack-reused 25812
Receiving objects: 100% (25902/25902), 15.84 MiB | 16.43 MiB/s, done.
Resolving deltas: 100% (18561/18561), done.
/content/flashlight
Note: switching to '035ead6efefb82b47c8c2e643603e87d38850076'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 035ead6e AdvancedIndex fix
/conte

In [8]:
# Hugging Face 서버에서 오디오 파일을 다운로드하고, 'tmp.wav'로 저장
!wget -O ./audio_samples/tmp.wav 'https://datasets-server.huggingface.co/assets/MLCommons/peoples_speech/--/dirty/train/0/audio/audio.wav'
# 설명:
# - 'wget'을 사용하여 지정된 URL에서 오디오 파일을 다운로드
# - '-O ./audio_samples/tmp.wav': 다운로드한 파일을 './audio_samples/' 디렉터리 내에 'tmp.wav'라는 이름으로 저장

# 다운로드된 오디오 파일의 샘플 레이트를 16000Hz로 변경하고, 'audio_noisy.wav'로 저장
!ffmpeg -y -i ./audio_samples/tmp.wav -ar 16000 ./audio_samples/audio_noisy.wav
# 설명:
# - 'ffmpeg'을 사용하여 오디오 파일 포맷 변환 및 샘플 레이트 조정
# - '-y': 출력 파일이 이미 존재할 경우 덮어쓰기를 강제함
# - '-i ./audio_samples/tmp.wav': 입력 파일 지정
# - '-ar 16000': 출력 파일의 오디오 샘플 레이트를 16000 Hz로 설정
# - './audio_samples/audio_noisy.wav': 변경된 샘플 레이트의 파일을 './audio_samples/' 디렉터리 내에 'audio_noisy.wav'라는 이름으로 저장


./audio_samples/tmp.wav: No such file or directory
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-l

In [11]:
# IPython 모듈을 임포트
import IPython

# './audio_samples/audio_noisy.wav' 경로의 오디오 파일을 재생
IPython.display.display(IPython.display.Audio("./audio_samples/audio_noisy.wav"))
# 설명:
# - `IPython.display.Audio`: IPython의 Audio 객체를 사용하여 오디오 파일을 로드
# - `IPython.display.display`: 로드된 오디오 객체를 노트북에서 재생할 수 있도록 출력

# 오디오 파일의 트랜스크립트 출력
print("Trancript: limiting emotions that we experience mainly in our childhood which stop us from living our life just open freedom i mean trust and")
# 설명:
# - `print` 함수를 사용하여 오디오 파일의 트랜스크립트를 출력
# - 이 텍스트는 해당 오디오 파일의 내용을 서면으로 표현한 것입니다.


ValueError: rate must be specified when data is a numpy array or list of audio samples.

In [ ]:
# os 모듈을 임포트하여 운영 체제 기능을 사용
import os

# 여러 환경 변수 설정
os.environ["TMPDIR"] = '/content/temp_dir'  # 임시 파일 저장 경로 설정
os.environ["PYTHONPATH"] = "."             # 현재 디렉터리를 파이썬 모듈 경로에 추가
os.environ["PREFIX"] = "INFER"             # 사용자 정의 환경 변수, 특정 목적으로 사용될 수 있음
os.environ["HYDRA_FULL_ERROR"] = "1"       # 오류 메시지를 전체적으로 출력하도록 설정
os.environ["USER"] = "micro"               # 사용자 환경 변수 설정, 'micro'로 정의

# 첫 번째 디코딩 방식을 표시하는 문자열 출력
print("======= WITHOUT LM DECODING=======")

# 언어 모델 없이 ASR 모델을 사용하여 오디오 파일을 디코딩
!python examples/mms/asr/infer/mms_infer.py --model "/content/fairseq/models_new/mms1b_fl102.pt" --lang "eng" \
--audio "/content/fairseq/audio_samples/audio.wav" "/content/fairseq/audio_samples/audio_noisy.wav"

# 두 번째 디코딩 방식을 시작하기 전에 구분 문자열 출력
print("\n\n\n======= WITH LM DECODING=======")

# 디코딩 명령어를 문자열로 설정
decoding_cmds = """
decoding.type=kenlm
decoding.beam=500
decoding.beamsizetoken=50
decoding.lmweight=2.69
decoding.wordscore=2.8
decoding.lmpath=/content/lmdecode/lm_common_crawl_small_4gram_prun0-6-15_200kvocab.bin
decoding.lexicon=/content/lmdecode/lexicon.txt
""".replace("\n", " ")  # 문자열에서 줄바꿈 제거

# 언어 모델을 사용하여 ASR 모델로 오디오 파일 디코딩, 추가 인자로 디코딩 명령어 사용
!python examples/mms/asr/infer/mms_infer.py --model "/content/fairseq/models_new/mms1b_fl102.pt" --lang "eng" \
--audio "/content/fairseq/audio_samples/audio.wav" "/content/fairseq/audio_samples/audio_noisy.wav" \
    --extra-infer-args '{decoding_cmds}'